In [1]:
import pandas as pd

TRUTH_TABLE_2 = pd.DataFrame([
    [str(x) for x in [0,0,0]],
    [str(x) for x in [0,1,0]],
    [str(x) for x in [1,0,0]],
    [str(x) for x in [1,1,1]],
])

TRUTH_TABLE_2.columns = ['A','B','Y']
TRUTH_TABLE_2

,A,B,Y
0,0,0,0
1,0,1,0
2,1,0,0
3,1,1,1


In [2]:
def gen_2_kmap(TRUTH_TABLE):
    kmap = [[-1 for _ in range(2)] for _ in range(2)]

    for a in range(2):
        for b in range(2):
            kmap[b][a] = TRUTH_TABLE[(TRUTH_TABLE['A']==str(a)) & (TRUTH_TABLE['B']==str(b))].iloc[0,-1]
    df = pd.DataFrame(kmap)
    df.index = ['0','1']
    df.columns = ['0','1']
    return df

def determine_state(var, value):
    if value=='1':
        return var
    else:
        return var+"\'"
    

In [3]:
def gen_state(var, value):
    state = ""
    for v, va in zip(var,value):
        state+=determine_state(v,va)+" & "

    return state[:-3]

def gen_boolean_map(vars, row_order, col_order):
    b_map = [["" for _ in range(len(col_order))] for _ in range(len(row_order))]
    for row_val in row_order:
        for col_val in col_order:
            b_map[row_order.index(row_val)][col_order.index(col_val)] = gen_state(vars,col_val+row_val)
    
    return pd.DataFrame(b_map)

def solve_kmap(kmap,bmap):
    m,n = kmap.shape
    expr = ""
    for i in range(m):
        for j in range(n):
            if(kmap.iloc[i,j]=="1"):
                expr += "(" + bmap.iloc[i,j] + ")" + " | "

    return expr[:-3]

def infix_to_postfix(expr):
    precedence = {'~':3, '&':2, '|':1}
    tokens = expr.replace('(', ' ( ').replace(')',' ) ').split()
    output, stack = [], []

    for t in tokens:
        if t.isalnum() or t.endswith("'"):
            if(t.endswith("'")):
                stack.append("~")
            output.append(t[0])
        
        elif t in precedence:
            while stack and stack[-1]!='(' and precedence[stack[-1]]>=precedence[t]:
                output.append(stack.pop())
            stack.append(t)
        
        elif t=='(':
            stack.append(t)
        elif t==')':
            while stack and stack[-1]!='(':
                output.append(stack.pop())
            stack.pop()
    
    while stack:
        output.append(stack.pop())
    return output

def postfix_to_verilog(postfix_stack, vars=['A','B']):
    stack = []
    for token in postfix_stack:
        if token.isalnum():
            stack.append(token)
        else:
            if(token=="~"):
                stack.append(token+stack.pop())
            else:
                b = stack.pop()
                stack.append("(" + stack.pop() + " " + token + " " + b + ")")
    
    inputs = ", ".join(f"input {x}" for x in vars)
    y = stack.pop()
    out = f"""module f({inputs}, output Y);
    assign Y = {y};
endmodule"""
    return out

def solve_truth_table_2(TRUTH_TABLE):
    kmap = gen_2_kmap(TRUTH_TABLE)
    bmap = gen_boolean_map("AB",['0','1'],['0','1'])
    expr = solve_kmap(kmap,bmap)
    code = postfix_to_verilog(infix_to_postfix(expr))

    return kmap, expr, code

In [4]:
print(solve_truth_table_2(TRUTH_TABLE_2)[2])

module f(input A, input B, output Y);
    assign Y = (A & B);
endmodule


In [5]:
TRUTH_TABLE_3 = pd.DataFrame([
    [str(x) for x in [0,0,0,1]],
    [str(x) for x in [0,0,1,0]],
    [str(x) for x in [0,1,0,1]],
    [str(x) for x in [0,1,1,0]],
    [str(x) for x in [1,0,0,1]],
    [str(x) for x in [1,0,1,0]],
    [str(x) for x in [1,1,0,1]],
    [str(x) for x in [1,1,1,1]]
])

TRUTH_TABLE_3.columns = ['A','B','C','Y']
TRUTH_TABLE_3

,A,B,C,Y
0,0,0,0,1
1,0,0,1,0
2,0,1,0,1
3,0,1,1,0
4,1,0,0,1
5,1,0,1,0
6,1,1,0,1
7,1,1,1,1


In [25]:
def map2(gray_order,a,b):
    return gray_order.index(str(a)+str(b))

def gen_3_kmap(TRUTH_TABLE):
    kmap = [[0 for _ in range(4)] for _ in range(2)]
    gray_code_order = ['00','01','11','10']
    for a in range(2):
        for b in range(2):
            for c in range(2):
                kmap[c][map2(gray_code_order,a,b)] = TRUTH_TABLE[(TRUTH_TABLE['A']==str(a)) & (TRUTH_TABLE['B']==str(b)) & (TRUTH_TABLE['C']==str(c))].iloc[0,-1]

    kmap_df = pd.DataFrame(kmap,index=['0','1'])
    kmap_df.columns = gray_code_order
    return kmap_df

def solve_truth_table_3(TRUTH_TABLE):
    kmap = gen_3_kmap(TRUTH_TABLE)
    bmap = gen_boolean_map("ABC",['0','1'],['00','01','11','10'])
    expr = solve_kmap(kmap,bmap)
    code = postfix_to_verilog(infix_to_postfix(expr),vars=['A','B','C'])

    return kmap,expr,code

print(solve_truth_table_3(TRUTH_TABLE_3)[2])

module f(input A, input B, input C, output Y);
    assign Y = ((((((~A & ~B) & ~C) | ((~A & B) & ~C)) | ((A & B) & ~C)) | ((A & ~B) & ~C)) | ((A & B) & C));
endmodule


In [17]:
TRUTH_TABLE_4 = pd.DataFrame([
    [str(a), str(b), str(c), str(d), 
     str((a and b) or (c and d))]   # function: Y = AB + CD
    for a in [0,1]
    for b in [0,1]
    for c in [0,1]
    for d in [0,1]
])

TRUTH_TABLE_4.columns = ['A','B','C','D','Y']
TRUTH_TABLE_4

,A,B,C,D,Y
0,0,0,0,0,0
1,0,0,0,1,0
2,0,0,1,0,0
3,0,0,1,1,1
4,0,1,0,0,0
5,0,1,0,1,0
6,0,1,1,0,0
7,0,1,1,1,1
8,1,0,0,0,0
9,1,0,0,1,0


In [ ]:
def gen_4_kmap(TRUTH_TABLE):
    kmap = [[0 for _ in range(4)] for _ in range(4)]
    gray_code_order = ['00','01','11','10']
    for a in range(2):
        for b in range(2):
            for c in range(2):
                for d in range(2):
                    kmap[map2(gray_code_order,c,d)][map2(gray_code_order,a,b)] = TRUTH_TABLE[(TRUTH_TABLE['A']==str(a)) & (TRUTH_TABLE['B']==str(b)) & (TRUTH_TABLE['C']==str(c)) & (TRUTH_TABLE['D']==str(d))].iloc[0,-1]

    kmap_df = pd.DataFrame(kmap,index=gray_code_order)
    kmap_df.columns = gray_code_order
    return kmap_df

def solve_truth_table_4(TRUTH_TABLE):
    kmap = gen_4_kmap(TRUTH_TABLE)
    bmap = gen_boolean_map('ABCD',['00','01','11','10'],['00','01','11','10'])
    expr = solve_kmap(kmap,bmap)
    code = postfix_to_verilog(infix_to_postfix(expr),vars='ABCD')

    return kmap, expr, code

print(solve_truth_table_4(TRUTH_TABLE_4)[1])

(A & B & C' & D') | (A & B & C' & D) | (A' & B' & C & D) | (A' & B & C & D) | (A & B & C & D) | (A & B' & C & D) | (A & B & C & D')


In [29]:
TRUTH_TABLE_5 = pd.DataFrame([
    [str(a), str(b), str(c), str(d), str(e), str((a and b) or (c and d and e))]
    for a in range(2)
    for b in range(2)
    for c in range(2)
    for d in range(2)
    for e in range(2)
], columns=['A','B','C','D','E','Y'])

print(TRUTH_TABLE_5)


    A  B  C  D  E  Y
0   0  0  0  0  0  0
1   0  0  0  0  1  0
2   0  0  0  1  0  0
3   0  0  0  1  1  0
4   0  0  1  0  0  0
5   0  0  1  0  1  0
6   0  0  1  1  0  0
7   0  0  1  1  1  1
8   0  1  0  0  0  0
9   0  1  0  0  1  0
10  0  1  0  1  0  0
11  0  1  0  1  1  0
12  0  1  1  0  0  0
13  0  1  1  0  1  0
14  0  1  1  1  0  0
15  0  1  1  1  1  1
16  1  0  0  0  0  0
17  1  0  0  0  1  0
18  1  0  0  1  0  0
19  1  0  0  1  1  0
20  1  0  1  0  0  0
21  1  0  1  0  1  0
22  1  0  1  1  0  0
23  1  0  1  1  1  1
24  1  1  0  0  0  1
25  1  1  0  0  1  1
26  1  1  0  1  0  1
27  1  1  0  1  1  1
28  1  1  1  0  0  1
29  1  1  1  0  1  1
30  1  1  1  1  0  1
31  1  1  1  1  1  1


In [ ]:
def solve_truth_table_5(TRUTH_TABLE):
    t0 = TRUTH_TABLE[TRUTH_TABLE['E']=='0'].reset_index(drop=True)
    t1 = TRUTH_TABLE[TRUTH_TABLE['E']=='1'].reset_index(drop=True)

    k0,e0,_ = solve_truth_table_4(t0)
    k1,e1,_ = solve_truth_table_4(t1)

    expr = f"(E' & {e0}) | (E & {e1})"
    code = postfix_to_verilog(infix_to_postfix(expr), vars=list("ABCDE"))

    return (k0,k1), expr, code

print(solve_truth_table_5(TRUTH_TABLE_5)[2])

module f(input A, input B, input C, input D, input E, output Y);
    assign Y = (((((~E & (((A & B) & ~C) & ~D)) | (((A & B) & ~C) & D)) | (((A & B) & C) & D)) | (((A & B) & C) & ~D)) | (((((((E & (((A & B) & ~C) & ~D)) | (((A & B) & ~C) & D)) | (((~A & ~B) & C) & D)) | (((~A & B) & C) & D)) | (((A & B) & C) & D)) | (((A & ~B) & C) & D)) | (((A & B) & C) & ~D)));
endmodule
